In [ ]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

# 비디오 캡쳐 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# 미디어파이프 (hands, drawing_utils, drawing_styles)객체 생성
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles
# 손 인식 기본값 설정
hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63*2,
    '12': 293.66*2,
    '11': 329.63*2,
    '10': 349.23*2,
    '9': 392.00*2,
    '8': 440.00*2,
    '7': 493.88*2,
    '6': 261.63*3,
    '5': 293.66*3,
    '4': 329.63*3,
    '3': 349.23*3,
    '2': 392.00*3,
    '1': 440.00*3,
    '0': 493.88*3,
}

# 음을 연주하는 함수
# winsound 에서 Beep 사용
def play_note(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 피아노 이미지 읽어오고 logo 변수에 할당
logo = cv.imread('./piano.png')
# cap.get(3)는 웹캠으로 가져오는 이미지의 가로 길이 (640)
frame_width = int(cap.get(3))
# cap.get(4)는 웹캠으로 가져오는 이미지의 세로 길이 (480)
frame_height = int(cap.get(4))
# 가져온 피아노이미지인 logo의 사이즈 설정
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
# logo를 위에서 설정한 사이즈로 resize
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14
# 계산된 x좌표들을 리스트로 담아둠
segment_coordinates = [i * segment_width for i in range(1, 14)]

# 인식 범위로 잡을 y값들을 지정
line_y1 = 380
line_y2 = 480

# 실시간으로 웹캠 화면에 피아노를 붙이고 손을 인식후 선택된 건반을 강조해주는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo
        
        # 웹캠으로 받아온 frame의 이미지속에서 손 인식
        res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
        # frame 이미지에서 손이 감지되었는지를 확인
        if res.multi_hand_landmarks: 
            # 감지된 손이 있으면 아래 반복문 실행
            for landmarks in res.multi_hand_landmarks:
                # 감지된 손 중에 필요한 좌표는 검지의 손끝 이므로 landmarks 중 8번째만 뽑아서 x,y로 저장
                # frame_width와 frame_height를 곱하는 이유는 landmarks 에서 감지된 좌표는 상대좌표이기 때문
                x = int(landmarks.landmark[8].x * frame_width)
                y = int(landmarks.landmark[8].y * frame_height) 
                
                # 탐지된 탐지된 손끝의 y좌표가 380 <= y <= 480 일경우 아래 실행
                if line_y1 <= y <= line_y2:
                    # 건반의 14개의 구역을 돌면서 어떠한 곳에서 탐지가 되었는지 찾아내는 과정
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            
                            # 탐지된 건반에 사각형 그리기 (노란색)
                            cv.rectangle(frame, (int(segment_coordinates[i]), 280),
                                            (int(segment_coordinates[i] + segment_width), line_y2),
                                            (0, 255, 255), -1)
                            break
            # 웹캠에서 프레임을 받아오지 못하면 종료
            if not ret:
                break
                
        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()
        
# 위의 함수를 계속 실행하기 위하여 스레드 설정
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

# 웹캠 화면에서 손을 인식하고 선택된 건반의 소리를 재생해주는 부분
while True:
    ret, frame = cap.read()

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks:
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)
            
            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                        
                        # 위와 같은 조건을 통하였을때 설정된 음계 실행
                        play_note(f'{i}', 0.3)
                        break
                    
    if cv.waitKey(1) == ord('q'):
        break


cap.release()
cv.destroyAllWindows()